In [267]:
from util import *
from sklearn.preprocessing import LabelEncoder,LabelBinarizer
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.svm import SVC
from mlxtend.classifier import StackingCVClassifier,StackingClassifier
from sklearn.multiclass import  OneVsOneClassifier,OneVsRestClassifier
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier,AdaBoostClassifier
from sklearn.preprocessing import StandardScaler,Normalizer
from sklearn.ensemble import VotingClassifier
pd.set_option('display.max_colwidth',1000)

In [268]:
train_all = load_train()
test_all = load_testA()
preprocess_basic_time(train_all)
preprocess_basic_wifi(train_all)
shop_info = load_shop_info()

In [286]:
mall_id = "m_1175" # 6587 ,9 21 36 72 77 86 
train = train_all[train_all.mall_id == mall_id]
# label
y = train.shop_id.values
le = LabelEncoder().fit(y)
y = le.transform(y)
#split
_train_index, _valid_index = get_last_one_week_index(train)
valid = train.iloc[_valid_index]
train = train.iloc[_train_index]
#wifi info
df, (train_index, train_use_wifi, train_matrix), (test_index, test_use_wifi, test_matrix) = get_wifi_cache2(mall_id)
train_wifi_all_x = train_matrix[_train_index]
valid_wifi_all_x = train_matrix[_valid_index]
valid_y = y[_valid_index]
train_y = y[_train_index]
train_lonlats = train[["longitude","latitude"]].values
valid_lonlats = valid[["longitude","latitude"]].values
train_wh = train[["weekday","hour"]].values
valid_wh = valid[["weekday","hour"]].values
train_w = train[["weekday"]].values
valid_w = valid[["weekday"]].values
train_h = train[["hour"]].values
valid_h = valid[["hour"]].values

train["dayofyear"] = train.dt.dt.dayofyear
valid["dayofyear"] = valid.dt.dt.dayofyear

indexs = choose_strong_wifi_index(-90,6,train_wifi_all_x)
train_x = np.concatenate([train_wifi_all_x[:,indexs],train_lonlats,train_wh],axis=1)
valid_x = np.concatenate([valid_wifi_all_x[:,indexs],valid_lonlats,valid_wh],axis=1)
lb = LabelBinarizer().fit(y)
train_b_y = lb.transform(train_y)
valid_b_y = lb.transform(valid_y)

In [287]:
tud = train.groupby(["user_id", "dayofyear"])["shop_id"].count().reset_index()
tud = tud[tud.shop_id >=2]
vud = valid.groupby(["user_id", "dayofyear"])["longitude"].count().reset_index()
vud = vud[vud.longitude >= 2]
train["minute"] = train.dt.dt.minute
train["hour_minute"] = train.hour * 60 + train.minute
valid["minute"] = valid.dt.dt.minute
valid["hour_minute"] = valid.hour * 60 + valid.minute
train["dayofyear"] = train.dt.dt.dayofyear
valid["dayofyear"] = valid.dt.dt.dayofyear
valid["sample_index"] = range(valid.shape[0])

In [288]:
#  多分类
rf_all = RandomForestClassifier(n_estimators=500,n_jobs=-1,class_weight="balanced")
train_all_x = np.concatenate([train_wifi_all_x[:, indexs], train_lonlats, train_wh],axis=1)
valid_all_x = np.concatenate([valid_wifi_all_x[:, indexs], valid_lonlats, valid_wh],axis=1)
rf_all.fit(train_all_x,train_y)
rf_all_pba = rf_all.predict_proba(valid_all_x)
acc(rf_all.predict(valid_all_x),valid_y)

0.9468840389166447

In [289]:
def get_top_k(pba,k = 3, min_proba = 0):
    _pba = pba.copy()
    def top_k(x,k):
        rs = []
        for _ in range(k):
            ind = np.argmax(x)
            if x[ind] < min_proba:
                continue
            rs.append(ind)
            x[ind] = 0
        return rs
    r = map(lambda x: top_k(x,k), _pba)
    return r

def acc_top_k(candidate, y):
    all_size = len(candidate)
    cor = 0
    for _can,_true in zip(candidate,y):
        if _true in _can:
            cor += 1
    return float(cor) / all_size


In [290]:
def corr_analyse_in_time(x, train, shops_cor_one_day,time=30): # 一天交易多次， 去某个商店在time分钟内可能去哪些商店
    shopids = train[(train.user_id == x[0]) & (train.dayofyear == x[1])][["shop_id","hour_minute"]].values
    for _i,_s in enumerate(shopids):
        for _j,_s2 in enumerate(shopids):
            if _i == _j:
                continue
            else:
                if abs(_s[1] - _s2[1]) > time:
                    continue
                if _s[0] not in shops_cor_one_day:
                    shops_cor_one_day[_s[0]] = {}
                if _s2[0] not in shops_cor_one_day[_s[0]]:
                    shops_cor_one_day[_s[0]][_s2[0]] = 0
                shops_cor_one_day[_s[0]][_s2[0]] += 1
    


def copy_candidate(old_candidate):
    new_candidate = []
    for _nda in old_candidate:
        new_candidate.append(_nda.copy())
    return new_candidate

            
def statistic_candidate(candidate, valid_y = None):
    candi_num = [len(_can) for _can in candidate]
    can_num = np.bincount(candi_num)
    correct = np.zeros((len(can_num),))
    if valid_y is not None:
        for _can, _true in zip(candidate,valid_y):
            if _true in _can:
                correct[len(_can)] += 1
        return can_num, correct.astype(int)
    else:
        return can_num,None  
    
def print_statistic_candidate(candidate, valid_y = None):
    can_num,cor_num = statistic_candidate(candidate, valid_y)
    print "all num",can_num
    if valid_y is not None:
        print "cor num",cor_num
        
def candidate_set(candidate):
    # 在all_rf产生的候选集合中用那一部分商场训练预测
    many = 0
    single = 0
    two = 0
    two_set = set()
    many_set = set()
    for _sample_index,_can in enumerate(candidate):
        if len(_can) ==1:
            single += 1
        else:
            if len(_can) == 2:
                two_set.add(",".join(sorted(list(_can.astype(str)))))
                two +=1
            else:
                many_set.add(",".join(sorted(list(_can.astype(str)))))
                many +=1
    print "many size",many
    print "two size",two
    print "single size",single
    print "two set length",len(two_set)
    print "many set length",len(many_set)
    return two_set,many_set


def forward_search(estimetor, trainx,trainy,fix_trainx,validx,validy,fix_validx):
    choose = []
    best_acc=0
    cc = 0
    while cc != -1:
        cc = -1
        for _i in range(trainx.shape[1]):
            if _i not in choose:
                curr = choose + [_i]
                ptrainx = np.concatenate([trainx[:,curr], fix_trainx],axis=1)
                pvalidx = np.concatenate([validx[:,curr], fix_validx],axis=1)
                m.fit(ptrainx,trainy)
                _acc = acc(m.predict(pvalidx),validy)
                if _acc > best_acc:
                    best_acc = _acc
                    cc = _i
        if cc != -1:
            print "choose", cc
            print "best acc", best_acc
            choose.append(cc)
        if best_acc == 1:
            break
    return choose

In [292]:
# 候选2 个, 最小概率0.02
candidate = get_top_k(rf_all_pba, 3, 0.02)
candidate = [rf_all.classes_.take(_can) for _can in candidate]
print acc_top_k(candidate, valid_y)

_ =candidate_set(candidate)

0.972258743098
many size 2175
two size 1368
single size 4063
two set length 166
many set length 554


In [293]:
shops_cor_10time = {}
_ = map(lambda x:corr_analyse_in_time(x,train, shops_cor_10time,time=10),tud[["user_id", "dayofyear"]].values)

shops_cor_60time = {}
_ = map(lambda x:corr_analyse_in_time(x,train, shops_cor_60time,time=60),tud[["user_id", "dayofyear"]].values)

shops_cor_1000time = {}
_ = map(lambda x:corr_analyse_in_time(x,train, shops_cor_1000time,time=1000),tud[["user_id", "dayofyear"]].values)




In [295]:
modify_candidate = copy_candidate(candidate)        
#一天内
_ = map(lambda x:remove_candidate(x,valid,shops_cor_10time,modify_candidate,le, stay=2, time = 20),vud[["user_id","dayofyear"]].values)
acc_top_k(modify_candidate, valid_y)

0.9722587430975546

In [296]:
print_statistic_candidate(candidate,valid_y)
print_statistic_candidate(modify_candidate,valid_y)
print_statistic_candidate(modify_candidate_u,valid_y)

all num [   0 4063 1368 2175]
cor num [   0 4061 1368 1966]
all num [   0 4063 1368 2175]
cor num [   0 4061 1368 1966]
all num [   0 1047  807  549  390  351  274  212  241  309 1199]
cor num [ 0 25 32 25 15 15 22  9 14 16 56]


In [284]:
modify_candidate_u = copy_candidate(candidate)
def remove_candidate_by_user(candidate,valid, train, deresase_to, le):
    u = train.groupby("user_id")["shop_id"].count().reset_index().sort_values("shop_id",ascending=False)
    u.rename(columns={"shop_id":"count"},inplace=True)
    uids = u[u["count"] >= (train.dayofyear.unique().shape[0] / 2)].user_id.values
    for _ind,_can in enumerate(candidate):
        if len(_can) > deresase_to:
            _uid = valid.iloc[_ind].user_id
            if _uid in uids:
                new_set = []
#                 print le.transform(valid.iloc[[_ind]]["shop_id"].values)
#                 print _can
                _max = np.bincount(le.transform(train[train.user_id == _uid].shop_id.values))
                
                while len(new_set) < deresase_to:
                    _m = np.argmax(_max)
                    if _max[_m] != 0:
                        if _m in _can:
                            new_set.append(_m)
                    else:
                        break
                    _max[_m] = 0
                
                if len(new_set) < deresase_to:
                    for _c in _can:
                        if _c not in new_set:
                            new_set.append(_c)
                        if len(new_set) == deresase_to:
                            break
                candidate[_ind] = np.asarray(new_set)
                
remove_candidate_by_user(modify_candidate_u, valid,train,deresase_to=2, le=le)
acc_top_k(modify_candidate_u, valid_y)

0.9663506227923406

In [136]:
len(can_set)

78

In [240]:
two_set,many_set = candidate_set(candidate)
print_statistic_candidate(candidate,valid_y)


many size 0
two size 2687
single size 1773
two set length 249
many set length 0
all num [   0 1773 2687]
cor num [   0 1771 2556]


In [197]:
most_set = set()
for _can in candidate:
    if len(_can) == 10:
        most_set.add(",".join(sorted(list(_can.astype(str)))))
print len(most_set)

245


In [198]:
most_dict = {}
most_dict_indexs={}
for _ind, _s in enumerate(most_set):
    if _ind % 20 == 0:
        print "{}/{}".format(_ind,len(most_set))
    _rf,_indexs = many_fit(_s,train_wifi_all_x,np.concatenate([train_lonlats,train_wh],axis=1), train_b_y,train_y)
    most_dict[_s] = _rf
    most_dict_indexs[_s] = _indexs

0/245
20/245
40/245
60/245
80/245
100/245
120/245
140/245
160/245
180/245
200/245
220/245
240/245


In [200]:
take = 5
most_predict = []
most_real = []
c = 0
s = 0
for _sample_index, _can in enumerate(candidate):
    if len(_can) == 10:
        s += 1
        _s = ",".join(sorted(list(_can.astype(str))))
        _indexs = most_dict_indexs[_s]
        _rf = most_dict[_s]
        pvalid_x = valid_wifi_all_x[[_sample_index]][:,_indexs] 
        pvalid_x = np.concatenate([pvalid_x, valid_lonlats[[_sample_index]],valid_wh[[_sample_index]]],axis=1)
        _p1 = _rf.predict_proba(pvalid_x)[0]
        ps=[]
        l = zip(_p1,_can)
        l = sorted(l,key=lambda x:-x[0])
        _ind = 0
        while(len(ps) != take):
            ps.append(l[_ind][1])
            _ind += 1
        most_predict.append(ps)
        most_real.append(valid_y[_sample_index])
#         print list(_can)
#         print ps
#         print valid_y[_sample_index]
        if valid_y[_sample_index] in list(_can[:take]):
            c+=1
#         print "==================="
print s
print acc_top_k(most_predict,most_real) * s
print c

264
98.0
192


In [100]:
# 找出30分钟内交易多次的根据训练集剔除不可能的候选集合
def remove_candidate(x,valid,shops_cor_time,modify_candidate, le, time = 30, stay = 3, continue_search=True):
    times = valid[(valid.user_id == x[0]) & (valid.dayofyear == x[1])][["hour_minute","sample_index"]].values
    for _i,_s in enumerate(times):
        for _j,_s2 in enumerate(times):
            if _i == _j:
                continue
            else:
                if abs(_s[0] - _s2[0]) > time:
                    continue
                if len(modify_candidate[_s[1]]) == 1:
                    if len(modify_candidate[_s2[1]]) == 1 or len(modify_candidate[_s2[1]]) == 2 or len(modify_candidate[_s2[1]]) == 3:
                        continue
                    _s_c = modify_candidate[_s[1]]
                    _ss_c = modify_candidate[_s2[1]]
                    sid = le.inverse_transform(_s_c)[0]
                    set1 = set(shops_cor_time[sid].keys())
                    set2 = set(le.inverse_transform(_ss_c))
                    sort =  zip(shops_cor_time[sid].keys(),shops_cor_time[sid].values())
                    sort =  sorted(sort,key=lambda x: -x[1])
                    
                    last_set = set()
                    _last_n = -1
                    
                    if continue_search:
                        search_sid = []
                        searched_sid = set()
                        searched_sid.add(sid)
                    
                    for _sn,_n in sort:
                        if _n == _last_n:# 和上一个数量相同的话
                            if _sn in set2:
                                last_set.add(_sn)
                                search_sid.append(_sn)
                                continue
                        # 和上一个数量不同了
                        if len(last_set) >= stay:
                            break
                        if _sn in set2:
                            last_set.add(_sn)
                            search_sid.append(_sn)
                            _last_n = _n
                            
                    # 从新加入的为基准继续搜索寻找
                    while continue_search and len(last_set) < stay and len(search_sid) != 0:
                        sid = search_sid.pop(0)
                        if sid in searched_sid:
                            continue
                        searched_sid.add(sid)
                        set1 = set(shops_cor_time[sid].keys())
                        sort =  zip(shops_cor_time[sid].keys(),shops_cor_time[sid].values())
                        sort =  sorted(sort,key=lambda x: -x[1])

                        _last_n = -1
                        for _sn,_n in sort:
                            if _n == _last_n:# 和上一个数量相同的话
                                if _sn in set2 and _sn not in last_set:
                                    last_set.add(_sn)
                                    search_sid.append(_sn)
                                    continue
                                    
                            # 和上一个数量不同了
                            if len(last_set) >= stay:
                                break
                                
                            if _sn in set2 and _sn not in last_set:
                                last_set.add(_sn)
                                search_sid.append(_sn)
                                _last_n = _n
                                
                    while len(last_set) < stay:
                        _can = modify_candidate[_s2[1]]
                        _can = le.inverse_transform(_can)
                        for _c in _can:
                            last_set.add(_c)
                            if len(last_set) == stay:
                                break
                        break
                                
                    modify_candidate[_s2[1]] = le.transform(list(last_set))

In [57]:
shops_cor_60time = {}
shops_cor_1000time = {}
shops_cor_30time = {}
_ = map(lambda x:corr_analyse_in_time(x,train, shops_cor_30time,time=30),tud[["user_id", "dayofyear"]].values)
_ = map(lambda x:corr_analyse_in_time(x,train, shops_cor_60time,time=60),tud[["user_id", "dayofyear"]].values)
_ = map(lambda x:corr_analyse_in_time(x,train, shops_cor_1000time,time=1000),tud[["user_id", "dayofyear"]].values)

In [101]:
modify_candidate = copy_candidate(modify_candidate_u)        
#一天内
_ = map(lambda x:remove_candidate(x,valid,shops_cor_1000time,modify_candidate,le, stay=6, time = 1000),vud[["user_id","dayofyear"]].values)
acc_top_k(modify_candidate, valid_y)

0.984304932735426

In [113]:
modify_candidate_h = copy_candidate(modify_candidate_u)
def remove_candidate_by_hour(candidate,valid,ht, deresase_to, le):
    for _ind,_can in enumerate(candidate):
        if len(_can) > deresase_to:
            _can = le.inverse_transform(_can)
            _h = valid.iloc[[_ind]]["hour"].values[0]
            _temp = ht[ht.hour==_h]["shop_id"].values
            new_set = []
            for _sn in _temp:
                if _sn in _can:
                    new_set.append(_sn)
                if len(new_set) >= deresase_to:
                    break
            if len(new_set) >= deresase_to:
                candidate[_ind] = le.transform(new_set)
                
remove_candidate_by_hour(modify_candidate_h, valid,ht,deresase_to=7, le=le)
acc_top_k(modify_candidate_h, valid_y)

0.9733183856502242

In [103]:
modify_candidate2 = copy_candidate(modify_candidate)        
#60分钟内
_ = map(lambda x:remove_candidate(x,valid,shops_cor_60time,modify_candidate2,le, stay=5, time = 60),vud[["user_id","dayofyear"]].values)
acc_top_k(modify_candidate2, valid_y)

0.9838565022421525

In [105]:
modify_candidate3 = copy_candidate(modify_candidate2)        
#30分钟内
_ = map(lambda x:remove_candidate(x,valid,shops_cor_30time,modify_candidate3,le, stay=3, time = 30),vud[["user_id","dayofyear"]].values)
acc_top_k(modify_candidate3, valid_y)

0.9827354260089686

In [111]:
print_statistic_candidate(candidate,valid_y)
print_statistic_candidate(modify_candidate_u,valid_y)
print_statistic_candidate(modify_candidate,valid_y)
print_statistic_candidate(modify_candidate2,valid_y)
print_statistic_candidate(modify_candidate3,valid_y)
print_statistic_candidate(modify_candidate4,valid_y)

all num [   0 1757  943  575  300  222  133  114   79   73  264]
cor num [   0 1755  942  575  298  222  132  114   78   69  211]
all num [   0 1757  943  599  294  218  130  112   78   70  259]
cor num [   0 1755  942  597  292  218  130  112   77   66  206]
all num [   0 1757  943  599  294  218  163  107   72   66  241]
cor num [   0 1755  942  597  292  218  156  105   71   62  192]
all num [   0 1757  943  599  294  252  130  106   72   66  241]
cor num [   0 1755  942  597  292  244  129  104   71   62  192]
all num [   0 1757  943  642  284  219  130  106   72   66  241]
cor num [   0 1755  942  630  283  215  129  104   71   62  192]
all num [   0 1757  943  642  284  219  130  452   11    6   16]
cor num [   0 1755  942  630  283  215  129  352   10    5   13]


In [146]:
two_set,many_set = candidate_set(candidate)

many size 1760
two size 943
single size 1757
two set length 109
many set length 958


In [229]:
# 2fenlei
two_dict = {}
two_dict_indexs = {}
for _ind,_s in enumerate(two_set):
    if _ind % 20 == 0:
        print "{}/{}".format(_ind,len(two_set))
    _ss = _s.split(",")
    s1 = int(_ss[0])
    s2 = int(_ss[1])
    
    s1_train = train[train_b_y[:,s1] == 1]
    s2_train = train[train_b_y[:,s2] == 1]
    s1_valid = valid[valid_b_y[:,s1] == 1]
    s2_valid = valid[valid_b_y[:,s2] == 1]

    s1_wifi_all_x = train_wifi_all_x[train_b_y[:,s1] == 1]
    s2_wifi_all_x = train_wifi_all_x[train_b_y[:,s2] == 1]
    s1_indexs = choose_strong_wifi_index(-115,6,s1_wifi_all_x)
    s2_indexs = choose_strong_wifi_index(-115,6,s2_wifi_all_x)
    _indexs = list(set(s1_indexs).union(set(s2_indexs)))
    _rf = RandomForestClassifier(n_jobs=-1,n_estimators=188,class_weight="balanced")
    _train_bool_index = (train_b_y[:,s1] == 1) | (train_b_y[:,s2]==1) 
    _valid_bool_index = (valid_b_y[:,s1] == 1) | (valid_b_y[:,s2]==1) 
    ptrain_x = train_wifi_all_x[_train_bool_index][:,_indexs] 
#     pvalid_x = valid_wifi_all_x[_valid_bool_index][:,_indexs] 
    ptrain_y = train_y[_train_bool_index] 
    ptrain_x = np.concatenate([ptrain_x, train_lonlats[_train_bool_index]],axis=1)
    _rf.fit(ptrain_x,ptrain_y)
    two_dict[_s] = _rf
    two_dict_indexs[_s] = _indexs


0/329
20/329
40/329
60/329
80/329
100/329
120/329
140/329
160/329
180/329
200/329
220/329
240/329
260/329
280/329
300/329
320/329


In [154]:
def many_fit(s, train_wifi_all_x, other_features, train_b_y, train_y):
    _ss = s.split(",")
    s1 = int(_ss[0])
    s1_wifi_all_x = train_wifi_all_x[train_b_y[:,s1] == 1]
    s1_indexs = choose_strong_wifi_index(-115,6,s1_wifi_all_x)
    _indexs = set(s1_indexs)
    _train_bool_index =  (train_b_y[:,s1] == 1)
    for _k in range(1,len(_ss)):
        s2 = int(_ss[_k])
        s2_wifi_all_x = train_wifi_all_x[train_b_y[:,s2] == 1]
        s2_indexs = choose_strong_wifi_index(-115,6,s2_wifi_all_x)
        _indexs = _indexs.union(set(s2_indexs))
        _train_bool_index =  _train_bool_index | (train_b_y[:,s2] == 1)
   
    _indexs = list(_indexs) 
    _rf = RandomForestClassifier(n_jobs=-1,n_estimators=288,class_weight="balanced")
    ptrain_x = train_wifi_all_x[_train_bool_index][:,_indexs] 
    ptrain_y = train_y[_train_bool_index] 
    ptrain_x = np.concatenate([ptrain_x, other_features[_train_bool_index]],axis=1)
    _rf.fit(ptrain_x,ptrain_y)
    return _rf, _indexs

In [34]:
#多分类
many_dict = {}
many_dict_indexs = {}
for _ind,_s in enumerate(many_set):
    if _ind % 20 == 0:
        print "{}/{}".format(_ind,len(many_set))
    _rf,_indexs = many_fit(_s,train_wifi_all_x,train_lonlats, train_b_y,train_y)
    many_dict[_s] = _rf
    many_dict_indexs[_s] = _indexs

0/951
20/951
40/951
60/951
80/951
100/951
120/951
140/951
160/951
180/951
200/951
220/951
240/951
260/951
280/951
300/951
320/951
340/951
360/951
380/951
400/951
420/951
440/951
460/951
480/951
500/951
520/951
540/951
560/951
580/951
600/951
620/951
640/951
660/951
680/951
700/951
720/951
740/951
760/951
780/951
800/951
820/951
840/951
860/951
880/951
900/951
920/951
940/951


In [230]:
last_predict,ones,twos,manys = predict_candidate(candidate,valid_y,two_dict,two_dict_indexs,many_dict,many_dict_indexs,2)


0
200
400
600
800
1000
1200
1400
1600
1800
2000
2200
2400
2600
2800
3000
3200
3400
3600
3800
4000
4200
4400


In [234]:
two_predict,two_real,two_error_sample_index,two_errors = twos
print "two", (np.asarray(two_predict) == np.asarray(two_real)).sum(), len(two_predict)
print "two error", Counter(two_errors)

two 3526 3751
two error Counter({'26,41': 32, '52,55': 18, '51,55': 18, '33,61': 9, '14,55': 8, '53,76': 6, '14,79': 5, '33,55': 5, '14,45': 5, '45,79': 4, '14,76': 4, '45,55': 4, '5,7': 3, '64,75': 3, '49,61': 3, '42,72': 3, '51,64': 3, '35,5': 3, '48,55': 3, '18,33': 3, '58,68': 2, '14,20': 2, '14,18': 2, '60,61': 2, '17,42': 2, '0,55': 2, '26,34': 2, '13,27': 2, '18,2': 2, '14,67': 2, '28,55': 2, '55,61': 2, '14,57': 2, '14,51': 2, '32,55': 2, '56,59': 2, '63,79': 1, '58,64': 1, '14,28': 1, '67,77': 1, '14,17': 1, '5,6': 1, '32,42': 1, '41,5': 1, '43,55': 1, '5,78': 1, '68,71': 1, '45,62': 1, '45,63': 1, '28,69': 1, '49,52': 1, '39,5': 1, '33,64': 1, '57,76': 1, '51,73': 1, '17,51': 1, '22,45': 1, '46,68': 1, '28,53': 1, '22,55': 1, '55,64': 1, '55,67': 1, '55,66': 1, '55,63': 1, '33,49': 1, '26,29': 1, '42,75': 1, '14,58': 1, '42,70': 1, '32,75': 1, '18,59': 1, '55,74': 1, '55,72': 1, '55,70': 1, '51,70': 1, '55,79': 1, '14,42': 1, '32,64': 1, '42,55': 1, '45,59': 1, '59,79': 1, '4

In [231]:
from collections import Counter
one_predict,one_real,one_error_sample_index,one_errors = ones

many_predict,many_real,many_error_sample_index,many_errors = manys
print "final acc", acc(np.asarray(last_predict),valid_y)
print "one", (np.asarray(one_predict) == np.asarray(one_real)).sum(), len(one_predict)
print "two", (np.asarray(two_predict) == np.asarray(two_real)).sum(), len(two_predict)
print "many", (np.asarray(many_predict) == np.asarray(many_real)).sum(), len(many_predict)
print "one error", Counter(one_errors)
print "two error", Counter(two_errors)
print "many error", Counter(many_errors)

final acc

AttributeError: 'bool' object has no attribute 'sum'

In [36]:
def predict_candidate(candidate, valid_y, two_dict, two_dict_indexes, many_dict, many_dict_indexs, predict_candi_length=-1):
    # 在all_rf产生的候选集合中用那一部分商场训练预测
    if predict_candi_length is None or predict_candi_length < 1:
        predict_candi_length = -1
    last_predict_by_candidates = []
    single_predict = []
    two_predict = []
    many_predict = []
    single_real = []
    two_real = []
    many_real = []
    one_error_sample_index = []
    two_error_sample_index = []
    many_error_sample_index = []
    one_errors = []
    two_errors = []
    many_errors = []
    for _sample_index,_can in enumerate(candidate):
        if _sample_index % 200 ==0:
            print _sample_index
        if len(_can) == 1 and (predict_candi_length == 1 or predict_candi_length == -1):
            last_predict_by_candidates.append(_can[0])
            single_predict.append(_can[0])
            single_real.append(valid_y[_sample_index])
            if _can[0] != [valid_y[_sample_index]]:
                one_errors.append(_can[0])
                one_error_sample_index.append(_sample_index)
        else:
            if len(_can) == 2 and (predict_candi_length == 2 or predict_candi_length == -1):
                s1 = _can[0]
                s2 = _can[1]
                _k = ",".join(sorted(list(_can.astype(str))))
                _indexs = two_dict_indexes[_k]
                _rf = two_dict[_k]
                pvalid_x = valid_wifi_all_x[[_sample_index]][:,_indexs] 
                pvalid_x = np.concatenate([pvalid_x, valid_lonlats[[_sample_index]]],axis=1)
                _p1 = _rf.predict(pvalid_x)[0]
                last_predict_by_candidates.append(_p1)
                two_predict.append(_p1)
                two_real.append(valid_y[_sample_index])
                if _p1 != valid_y[_sample_index]:
                    two_errors.append(",".join(sorted(list(_can.astype(str)))))
                    two_error_sample_index.append(_sample_index)
            elif len(_can) > 2 and (predict_candi_length > 2 or predict_candi_length == -1):
                _s = ",".join(sorted(list(_can.astype(str))))
                _indexs = many_dict_indexs[_s]
                _rf = many_dict[_s]
                pvalid_x = valid_wifi_all_x[[_sample_index]][:,_indexs] 
                pvalid_x = np.concatenate([pvalid_x, valid_lonlats[[_sample_index]]],axis=1)
                _p1 = _rf.predict(pvalid_x)[0]
                last_predict_by_candidates.append(_p1)
                many_predict.append(_p1)
                many_real.append(valid_y[_sample_index])
                if _p1 != valid_y[_sample_index]:
                    many_errors.append(_s)
                    many_error_sample_index.append(_sample_index)
    return last_predict_by_candidates, (single_predict,single_real,one_error_sample_index,one_errors),(two_predict,two_real,two_error_sample_index,two_errors),(many_predict,many_real,many_error_sample_index,many_errors)         
